# Reading and cleaning bike trip history data

This notebook includes initial exploration of the bike trip history data. Specifically, it was used to develop doe for reading tripdata.zip files and extract counts of trips by day.

In [18]:
import itertools
import pandas as pd
import numpy as np

First, when reading the files, it being apparent (from runtimes) that the pandas datetime parser was slow due to inconsistant formatting of the starttime field.

In [19]:
years = range(2014,2017)
months = [str(x).zfill(2) for x in range(1,13)]

In [35]:
for year, month in itertools.product(years, months):
    df = pd.read_csv('./../data/raw/{}{}-citibike-tripdata.zip'.format(year,month), usecols=[1])
    print df.head(2)
    print df.tail(2)
    print '\n'

             starttime
0  2014-01-01 00:00:06
1  2014-01-01 00:00:38
                  starttime
300398  2014-01-31 23:58:34
300399  2014-01-31 23:58:43


             starttime
0  2014-02-01 00:00:00
1  2014-02-01 00:00:03
                  starttime
224734  2014-02-28 23:59:10
224735  2014-02-28 23:59:47


             starttime
0  2014-03-01 00:00:16
1  2014-03-01 00:00:57
                  starttime
439115  2014-03-31 23:59:40
439116  2014-03-31 23:59:54


             starttime
0  2014-04-01 00:00:07
1  2014-04-01 00:00:20
                  starttime
670778  2014-04-30 23:56:43
670779  2014-04-30 23:57:28


             starttime
0  2014-05-01 00:08:32
1  2014-05-01 00:10:16
                  starttime
866115  2014-05-31 23:59:54
866116  2014-05-31 23:59:59


             starttime
0  2014-06-01 00:00:02
1  2014-06-01 00:00:27
                  starttime
936878  2014-06-30 23:59:50
936879  2014-06-30 23:59:50


             starttime
0  2014-07-01 00:00:04
1  2014-07-01 00:00:06
 

Our strategy to deal with this will be two fold- read in the first value, then
    - Match the '-' to decide on the datetime pattern.
    - If no match, count the number of ":" to specify pattern.

In [40]:
for year, month in itertools.product(years, months):
    sample = pd.read_csv('./../data/raw/{}{}-citibike-tripdata.zip'.format(year,month), usecols=[1], nrows=1)
    col = sample.columns[0]
    sample = sample.loc[0][0]
    if sample.find('-') != -1:
        format='%Y-%m-%d %H:%M:%S'
    else:
        if sample.count(':') == 1:
            format='%m/%d/%Y %H:%M'
        elif sample.count(':') == 2:
            format='%m/%d/%Y %H:%M:%S'
    df = pd.read_csv('./../data/raw/{}{}-citibike-tripdata.zip'.format(year,month), usecols=[1], nrows=100)
    df[col] =  pd.to_datetime(df[col], format=format)
    
    print year, month
    print df.info()
    print '\n'

2014 01
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 1 columns):
starttime    100 non-null datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 872.0 bytes
None


2014 02
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 1 columns):
starttime    100 non-null datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 872.0 bytes
None


2014 03
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 1 columns):
starttime    100 non-null datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 872.0 bytes
None


2014 04
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 1 columns):
starttime    100 non-null datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 872.0 bytes
None


2014 05
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 1 columns):
starttime    100 non-null datetime64[ns]
dtyp

Next we use a sample file to develop aggregation code

In [43]:
df = pd.read_csv('./../data/raw/{}{}-citibike-tripdata.zip'.format(2016,12), parse_dates=[0],usecols=[1])

In [44]:
df['Date'] = df['Start Time'].dt.floor('d')
df.drop('Start Time', axis=1, inplace=True)

In [45]:
df = df.groupby('Date').size().to_frame('Trip Count')

In [46]:
df['Day of Week'] = df.index.dayofweek
df['Month'] = df.index.month

In [47]:
df.head()

,Trip Count,Day of Week,Month
Date,,,
2016-12-01,43242,3,12
2016-12-02,42371,4,12
2016-12-03,29854,5,12
2016-12-04,27455,6,12
2016-12-05,35932,0,12
